# Sanity Check For Click-based Lambdas

In [ ]:
import os
os.system("export LC_ALL=en_US.UTF-8")
os.system("export LANG=en_US.UTF-8")

%matplotlib inline

import numpy as np
np.set_printoptions(precision=2, linewidth=np.inf, suppress=True)
np.seterr(invalid='ignore', divide='ignore')

import cPickle as pickle
import matplotlib.pyplot as plt
from matplotlib.colors import hsv_to_rgb

from matplotlib.ticker import MultipleLocator

from ipywidgets import interact, interactive, Dropdown, FloatText, VBox, HBox

from IPython.display import display

# Load the click models for the queries of interest.
# with open('./data/model_query_uniform_lambdas_v1_collection_c2.pkl') as ifile:
# with open('./data/model_query_softmax_lambdas_v1_collection_c2.pkl') as ifile:
# with open('./data/model_query_uniform_lambdas_v2_collection_c2.pkl') as ifile:
with open('./data/model_query_softmax_lambdas_v2_collection_c2.pkl') as ifile:
# with open('./data/model_query_uniform_lambdas_v3_collection_c2.pkl') as ifile:
# with open('./data/model_query_softmax_lambdas_v3_collection_c2.pkl') as ifile:
    MQD = pickle.load(ifile)

# For reproducibility -- re-seed the click models' RNGs.
for click_model_type in MQD:
    for query in MQD[click_model_type]:
        MQD[click_model_type][query]['model'].seed = 42

# Available Click Models

In [ ]:
MQD.keys()

# MSE Error In Delta Estimation

In [ ]:
query_model_losses = {}
for query in MQD['CM']:
    model_losses = {}
    for click_model_name in MQD:
        model = MQD[click_model_name][query]['model']
        relevances = MQD[click_model_name][query]['relevances']
        n_documents = len(relevances)

        test_lambdas = np.zeros((n_documents, n_documents), dtype='float64')

        if click_model_name == 'CM':
            p_attract = model.click_proba

            for i in range(n_documents):
                for j in range(n_documents):
                    if i == j: continue
                    test_lambdas[i, j] = (1.0 - p_attract[j]) * p_attract[i]

        elif click_model_name == 'PBM':
            p_attract = model.click_proba
            p_exam = model.exam_proba

            for i in range(n_documents):
                for j in range(n_documents):
                    if i == j: continue
                    test_lambdas[i, j] = (1.0 - p_exam[0] * p_attract[j]) * p_exam[1] * p_attract[i]

        elif click_model_name == 'CCM':
            p_attract = model.p_attraction
            tau1 = 1.0 - model.p_stop_noclick
            tau2 = 1.0 - model.p_stop_click_norel
            tau3 = 1.0 - model.p_stop_click_rel

            for i in range(n_documents):
                for j in range(n_documents):
                    if i == j: continue
                    test_lambdas[i, j] = (1.0 - p_attract[j]) * tau1 * p_attract[i]

        elif click_model_name == 'DBN':
            p_attract = model.click_proba
            p_cont = 1.0 - model.stop_proba
            p_abandon = model.abandon_proba

            for i in range(n_documents):
                for j in range(n_documents):
                    if i == j: continue
                    test_lambdas[i, j] = (1.0 - p_attract[j]) * p_attract[i]

        elif click_model_name == 'DCM':
            p_attraction = model.click_proba
            p_cont = 1.0 - model.stop_proba

            for i in range(n_documents):
                for j in range(n_documents):
                    if i == j: continue
                    test_lambdas[i, j] = (1.0 - p_attraction[j]) * p_attraction[i]

        elif click_model_name == 'UBM':
            p_attraction = model.p_attraction
            p_examination = model.p_examination

            for i in range(n_documents):
                for j in range(n_documents):
                    if i == j: continue
                    test_lambdas[i, j] = (1.0 - p_attraction[j]) * p_examination[1, -1] * p_attraction[i]

        else:
            raise ValueError('unknow click model: %s' % click_model_name)

        # This is what we want to estimate with click lambdas?
        test_lambdas = test_lambdas - test_lambdas.T

        viewed_loss, total_loss = np.empty(10), np.empty(10)
        for i in range(10):
            viewed_lambdas = MQD[click_model_name][query]['stats'][1000000]['viewed_lambdas'][i]
            total_lambdas = MQD[click_model_name][query]['stats'][1000000]['total_lambdas'][i]

            viewed_loss[i] = np.mean((test_lambdas - viewed_lambdas)**2)
            total_loss[i] = np.mean((test_lambdas - total_lambdas)**2)
        
        model_losses[click_model_name] = (np.mean(viewed_loss), np.std(viewed_loss),
                                          np.mean(total_loss), np.std(total_loss))
    query_model_losses[query] = model_losses

print '   QUERY     MODEL     VL/MSE       VL/STD       TL/MSE       TL/STD'
print '  -------   -------   --------     --------     --------     --------'
for click_model_name in query_model_losses['2548']:
    for query in query_model_losses:
        params = (query, click_model_name) + query_model_losses[query][click_model_name]
        print '{0:>8s} {1:>8s} {2:>12.6f} {3:>12.6f} {4:>12.6f} {5:>12.6f}'.format(*params)

# Click Model

In [ ]:
click_model_name = 'UBM'

printed_model_characteristics = False

for query in MQD[click_model_name]:
    model = MQD[click_model_name][query]['model']
    relevances = MQD[click_model_name][query]['relevances']
    n_documents = len(relevances)
    
    deltas = relevances[:, None] - relevances[None, :]
    test_lambdas = np.zeros((n_documents, n_documents), dtype='float64')

    if click_model_name == 'CM':
        p_attract = model.click_proba    
        
        if not printed_model_characteristics:
            printed_model_characteristics = True
            
            print 'CM model'
            print '--------'
            print 'Attractiveness:', p_attract
            print
            
        for i in range(n_documents):
            for j in range(n_documents):
                if i == j: continue
                test_lambdas[i, j] = (1.0 - p_attract[j]) * p_attract[i]

    elif click_model_name == 'PBM':
        p_attract = model.click_proba
        p_exam = model.exam_proba
        
        if not printed_model_characteristics:
            printed_model_characteristics = True
            
            print 'PBM model'
            print '---------'
            print 'Attractiveness:', p_attract
            print 'Examination:   ', p_exam
            print

        for i in range(n_documents):
            for j in range(n_documents):
                if i == j: continue
                test_lambdas[i, j] = (1.0 - p_exam[0] * p_attract[j]) * p_exam[1] * p_attract[i]

    elif click_model_name == 'CCM':
        p_attract = model.p_attraction
        tau1 = 1.0 - model.p_stop_noclick
        tau2 = 1.0 - model.p_stop_click_norel
        tau3 = 1.0 - model.p_stop_click_rel
        
        if not printed_model_characteristics:
            printed_model_characteristics = True
            
            print 'PBM model'
            print '---------'
            print 'Attractiveness:', p_attract
            print 'Tau1:          ', tau1
            print 'Tau2:          ', tau2
            print 'Tau3:          ', tau3
            print

        for i in range(n_documents):
            for j in range(n_documents):
                if i == j: continue
                test_lambdas[i, j] = (1.0 - p_attract[j]) * tau1 * p_attract[i]

    elif click_model_name == 'DBN':
        p_attract = model.click_proba
        p_cont = 1.0 - model.stop_proba
        p_abandon = model.abandon_proba
        
        if not printed_model_characteristics:
            printed_model_characteristics = True
            
            print 'DBN model'
            print '---------'
            print 'Attractiveness:', p_attract
            print 'Continuation:  ', p_cont
            print 'Abandonment:   ', p_abandon

        for i in range(n_documents):
            for j in range(n_documents):
                if i == j: continue
                test_lambdas[i, j] = (1.0 - p_attract[j]) * p_attract[i]

    elif click_model_name == 'DCM':
        p_attraction = model.click_proba
        p_cont = 1.0 - model.stop_proba
        
        if not printed_model_characteristics:
            printed_model_characteristics = True
            
            print 'DCM model'
            print '---------'
            print 'Attractiveness:', p_attraction
            print 'Continuation:  ', p_cont
            print

        for i in range(n_documents):
            for j in range(n_documents):
                if i == j: continue
                test_lambdas[i, j] = (1.0 - p_attraction[j]) * p_attraction[i]
        
    elif click_model_name == 'UBM':
        p_attraction = model.p_attraction
        p_examination = model.p_examination
        
        if not printed_model_characteristics:
            printed_model_characteristics = True
            
            print 'UBM model'
            print '---------'
            print 'Attractiveness:', p_attraction
            print 'Examination:  '
            print p_examination[:,[9] + range(9)].T
            print

        for i in range(n_documents):
            for j in range(n_documents):
                if i == j: continue
                test_lambdas[i, j] = (1.0 - p_attraction[j]) * p_examination[1, -1] * p_attraction[i]

    else:
        raise ValueError('unknow click model: %s' % click_model_name)

    # This is what we want to estimate with click lambdas?
    test_lambdas = test_lambdas - test_lambdas.T

    viewed_loss, total_loss = np.empty(10), np.empty(10)
    for i in range(10):
        viewed_lambdas = MQD[click_model_name][query]['stats'][1000000]['viewed_lambdas'][i]
        total_lambdas = MQD[click_model_name][query]['stats'][1000000]['total_lambdas'][i]
    
        viewed_loss[i] = np.mean((test_lambdas - viewed_lambdas)**2)
        total_loss[i] = np.mean((test_lambdas - total_lambdas)**2)        
    
    lambdas = MQD[click_model_name][query]['stats'][1000000]['lambdas'][0]
    
    viewed_lambdas = MQD[click_model_name][query]['stats'][1000000]['viewed_lambdas'][0]
    viewed_counts = MQD[click_model_name][query]['stats'][1000000]['viewed_counts'][0]
    
    total_lambdas = MQD[click_model_name][query]['stats'][1000000]['total_lambdas'][0]
    total_counts = MQD[click_model_name][query]['stats'][1000000]['total_counts'][0]
    
    print 'deltas:'
    print '-------'
    print deltas
    print
    print 'test lambdas:'
    print '-------------'
    print test_lambdas
    print 
    print 'viewed lambdas'
    print '--------------'
    print 'MSE: %.6f (+/- %.6f SE)' % (np.mean(viewed_loss), np.std(viewed_loss))
    print viewed_lambdas
    print
    print 'total lambdas'
    print '--------------'
    print 'MSE: %.6f (+/- %.6f SE)' % (np.mean(total_loss), np.std(total_loss))
    print total_lambdas
    print
    print 'test lambdas / deltas:'
    print '----------------------'
    print np.nan_to_num(test_lambdas / deltas)
    print
    print 'viewed lambdas / deltas:'
    print '----------------------'
    print np.nan_to_num(viewed_lambdas / deltas)
    print
    print 'total lambdas / deltas:'
    print '----------------------'
    print np.nan_to_num(total_lambdas / deltas)
    print
    print 'lambdas:'
    print lambdas
    print
    print 'total counts:'
    print '-------------'
    print total_counts
    print
    print 'viewed counts:'
    print '--------------'
    print viewed_counts
    print
    print 'viewed / total counts:'
    print '----------------------'
    print np.nan_to_num(1.0 * viewed_counts / total_counts)
    break
            

# Probabilities of Ranking-Click Patterns for UBM model

## Helper Code Generating Ranking-Click Paterns and Their Probabilities

In [ ]:
import numpy as np
from itertools import permutations

def generate_clicks(n):
    i = 0
    while i < (1 << n):
        cs = []
        for j in range(n):
            cs.append(1 if (i & (1 << (n - 1 - j))) > 0 else 0)
        yield cs
        i += 1

def generate_latex_table_header():
    return r'\begin{array}{|c|c|l|}' + '\n' + r'\hline\hline'

def generate_latex_table_body(ranking, clicks):
    last_c_r = 0

    if ranking.index('y') < ranking.index('x'):
        return None

    result = ''.join(ranking) + ' & ' + ''.join(map(str, clicks)) + ' & '

    for r, c_r in enumerate(clicks):
        if c_r == 1:
            result += r'\gamma_{%d,%d}\alpha_{%c}' % (last_c_r, r + 1, ranking[r])
            last_c_r = r + 1
        else:
            result += r'(1 - \gamma_{%d,%d}\alpha_{%c})' % (last_c_r, r + 1, ranking[r])
        if r + 1 < len(clicks):
            result += r' \cdot '
    result += r'\\'
    return result

def generate_latex_table_footer():
    return r'\hline\hline' + '\n' + r'\end{array}'

rankings = list(permutations('xyz'))
clicks = list(generate_clicks(3))

print generate_latex_table_header()

for i, r in enumerate(rankings):
    for c in clicks:
        s = generate_latex_table_body(r, c)
        if s is not None:
            print s
    if s is not None and i + 1 < len(rankings):
        print '\hline'

print generate_latex_table_footer()

$$
\begin{array}{|c|c|l|}
\hline\hline
xyz & 000 & (1 - \gamma_{0,1}\alpha_{x}) \cdot (1 - \gamma_{0,2}\alpha_{y}) \cdot (1 - \gamma_{0,3}\alpha_{z})\\
xyz & 001 & (1 - \gamma_{0,1}\alpha_{x}) \cdot (1 - \gamma_{0,2}\alpha_{y}) \cdot \gamma_{0,3}\alpha_{z}\\
xyz & 010 & (1 - \gamma_{0,1}\alpha_{x}) \cdot \gamma_{0,2}\alpha_{y} \cdot (1 - \gamma_{2,3}\alpha_{z})\\
xyz & 011 & (1 - \gamma_{0,1}\alpha_{x}) \cdot \gamma_{0,2}\alpha_{y} \cdot \gamma_{2,3}\alpha_{z}\\
xyz & 100 & \gamma_{0,1}\alpha_{x} \cdot (1 - \gamma_{1,2}\alpha_{y}) \cdot (1 - \gamma_{1,3}\alpha_{z})\\
xyz & 101 & \gamma_{0,1}\alpha_{x} \cdot (1 - \gamma_{1,2}\alpha_{y}) \cdot \gamma_{1,3}\alpha_{z}\\
xyz & 110 & \gamma_{0,1}\alpha_{x} \cdot \gamma_{1,2}\alpha_{y} \cdot (1 - \gamma_{2,3}\alpha_{z})\\
xyz & 111 & \gamma_{0,1}\alpha_{x} \cdot \gamma_{1,2}\alpha_{y} \cdot \gamma_{2,3}\alpha_{z}\\
\hline
xzy & 000 & (1 - \gamma_{0,1}\alpha_{x}) \cdot (1 - \gamma_{0,2}\alpha_{z}) \cdot (1 - \gamma_{0,3}\alpha_{y})\\
xzy & 001 & (1 - \gamma_{0,1}\alpha_{x}) \cdot (1 - \gamma_{0,2}\alpha_{z}) \cdot \gamma_{0,3}\alpha_{y}\\
xzy & 010 & (1 - \gamma_{0,1}\alpha_{x}) \cdot \gamma_{0,2}\alpha_{z} \cdot (1 - \gamma_{2,3}\alpha_{y})\\
xzy & 011 & (1 - \gamma_{0,1}\alpha_{x}) \cdot \gamma_{0,2}\alpha_{z} \cdot \gamma_{2,3}\alpha_{y}\\
xzy & 100 & \gamma_{0,1}\alpha_{x} \cdot (1 - \gamma_{1,2}\alpha_{z}) \cdot (1 - \gamma_{1,3}\alpha_{y})\\
xzy & 101 & \gamma_{0,1}\alpha_{x} \cdot (1 - \gamma_{1,2}\alpha_{z}) \cdot \gamma_{1,3}\alpha_{y}\\
xzy & 110 & \gamma_{0,1}\alpha_{x} \cdot \gamma_{1,2}\alpha_{z} \cdot (1 - \gamma_{2,3}\alpha_{y})\\
xzy & 111 & \gamma_{0,1}\alpha_{x} \cdot \gamma_{1,2}\alpha_{z} \cdot \gamma_{2,3}\alpha_{y}\\
\hline
zxy & 000 & (1 - \gamma_{0,1}\alpha_{z}) \cdot (1 - \gamma_{0,2}\alpha_{x}) \cdot (1 - \gamma_{0,3}\alpha_{y})\\
zxy & 001 & (1 - \gamma_{0,1}\alpha_{z}) \cdot (1 - \gamma_{0,2}\alpha_{x}) \cdot \gamma_{0,3}\alpha_{y}\\
zxy & 010 & (1 - \gamma_{0,1}\alpha_{z}) \cdot \gamma_{0,2}\alpha_{x} \cdot (1 - \gamma_{2,3}\alpha_{y})\\
zxy & 011 & (1 - \gamma_{0,1}\alpha_{z}) \cdot \gamma_{0,2}\alpha_{x} \cdot \gamma_{2,3}\alpha_{y}\\
zxy & 100 & \gamma_{0,1}\alpha_{z} \cdot (1 - \gamma_{1,2}\alpha_{x}) \cdot (1 - \gamma_{1,3}\alpha_{y})\\
zxy & 101 & \gamma_{0,1}\alpha_{z} \cdot (1 - \gamma_{1,2}\alpha_{x}) \cdot \gamma_{1,3}\alpha_{y}\\
zxy & 110 & \gamma_{0,1}\alpha_{z} \cdot \gamma_{1,2}\alpha_{x} \cdot (1 - \gamma_{2,3}\alpha_{y})\\
zxy & 111 & \gamma_{0,1}\alpha_{z} \cdot \gamma_{1,2}\alpha_{x} \cdot \gamma_{2,3}\alpha_{y}\\
\hline\hline
\end{array}
$$

## X skipped and Y clicked
Binary random variable $\lambda_{y,x}$ is 1 only if the document $x$ was presented above document $y$ and the user clicked on document $y$. The following table show ranking-click paterns consistent with $\lambda_{y,x} = 1$:

$$
\begin{array}{|c|c|l|}
\hline\hline
xyz & 010 & (1 - \gamma_{0,1}\alpha_{x}) \cdot \gamma_{0,2}\alpha_{y} \cdot (1 - \gamma_{2,3}\alpha_{z})\\
xyz & 011 & (1 - \gamma_{0,1}\alpha_{x}) \cdot \gamma_{0,2}\alpha_{y} \cdot \gamma_{2,3}\alpha_{z}\\
\hline
xzy & 001 & (1 - \gamma_{0,1}\alpha_{x}) \cdot (1 - \gamma_{0,2}\alpha_{z}) \cdot \gamma_{0,3}\alpha_{y}\\
xzy & 011 & (1 - \gamma_{0,1}\alpha_{x}) \cdot \gamma_{0,2}\alpha_{z} \cdot \gamma_{2,3}\alpha_{y}\\
\hline
zxy & 001 & (1 - \gamma_{0,1}\alpha_{z}) \cdot (1 - \gamma_{0,2}\alpha_{x}) \cdot \gamma_{0,3}\alpha_{y}\\
zxy & 101 & \gamma_{0,1}\alpha_{z} \cdot (1 - \gamma_{1,2}\alpha_{x}) \cdot \gamma_{1,3}\alpha_{y}\\
\hline\hline
\end{array}
$$

Cosidering that each ranking (permutation) has the same probability of being shown, we get the following expression for $\lambda_{y,x}$:

\begin{array}{}
P(\lambda_{y,x} = 1\vert\{x,y,z\}) = &\gamma_{0,2}\alpha_{y} - \gamma_{0,1}\gamma_{0,2}\alpha_{x}\alpha_{y} + \gamma_{0,3}\alpha_{y} - \gamma_{0,2}\gamma_{0,3}\alpha_{y}\alpha_{z} - \gamma_{0,1}\gamma_{0,3}\alpha_{x}\alpha_{y} + \\
&\gamma_{0,1}\gamma_{0,2}\gamma_{0,3}\alpha_{x}\alpha_{y}\alpha_{z} + \gamma_{0,2}\gamma_{2,3}\alpha_{y}\alpha_{z} - \gamma_{0,1}\gamma_{0,2}\gamma_{2,3}\alpha_{x}\alpha_{y}\alpha_{z} + \gamma_{0,3}\alpha_{y} - \gamma_{0,2}\gamma_{0,3}\alpha_{x}\alpha_{y} - \\
&\gamma_{0,1}\gamma_{0,3}\alpha_{y}\alpha_{z} + \gamma_{0,1}\gamma_{0,2}\gamma_{0,3}\alpha_{x}\alpha_{y}\alpha_{z} + \gamma_{0,1}\gamma_{1,3}\alpha_{y}\alpha_{z} - \gamma_{0,1}\gamma_{1,2}\gamma_{1,3}\alpha_{x}\alpha_{y}\alpha_{z}
\end{array}

by summing up a few terms in above expression we get

\begin{array}{}
P(\lambda_{y,x} = 1\vert\{x,y,z\}) = &(\gamma_{0,2} + 2\gamma_{0,3})\alpha_{y} - (\gamma_{0,1}\gamma_{0,2} + \gamma_{0,1}\gamma_{0,3} + \gamma_{0,2}\gamma_{0,3})\alpha_{x}\alpha_{y} - (\gamma_{0,1}\gamma_{0,3} + \gamma_{0,2}\gamma_{0,3} - \gamma_{0,2}\gamma_{2,3} - \gamma_{0,1}\gamma_{1,3})\alpha_{y}\alpha_{z} + \\
&(\gamma_{0,1}\gamma_{0,2}\gamma_{0,3} - \gamma_{0,1}\gamma_{0,2}\gamma_{2,3} + \gamma_{0,1}\gamma_{0,2}\gamma_{0,3} - \gamma_{0,1}\gamma_{1,2}\gamma_{1,3})\alpha_{x}\alpha_{y}\alpha_{z}
\end{array}

In more generality we may consider the case in which the rankings are sampled from a non-uniform distribution $P(xyz)$ which denotes the probability of seeing the permutation $xyz$.

\begin{array}{}
P(\lambda_{y,x} = 1\vert\{x,y,z\}) = &\gamma_{0,2}\alpha_{y}P(xyz) - \gamma_{0,1}\gamma_{0,2}\alpha_{x}\alpha_{y}P(xyz) + \gamma_{0,3}\alpha_{y}P(xzy) - \gamma_{0,2}\gamma_{0,3}\alpha_{y}\alpha_{z}P(xzy) - \gamma_{0,1}\gamma_{0,3}\alpha_{x}\alpha_{y}P(xzy) + \\
&\gamma_{0,1}\gamma_{0,2}\gamma_{0,3}\alpha_{x}\alpha_{y}\alpha_{z}P(xzy) + \gamma_{0,2}\gamma_{2,3}\alpha_{y}\alpha_{z}P(xzy) - \gamma_{0,1}\gamma_{0,2}\gamma_{2,3}\alpha_{x}\alpha_{y}\alpha_{z}P(xzy) + \gamma_{0,3}\alpha_{y}P(zxy) - \gamma_{0,2}\gamma_{0,3}\alpha_{x}\alpha_{y}P(zxy) - \\
&\gamma_{0,1}\gamma_{0,3}\alpha_{y}\alpha_{z}P(zxy) + \gamma_{0,1}\gamma_{0,2}\gamma_{0,3}\alpha_{x}\alpha_{y}\alpha_{z}P(zxy) + \gamma_{0,1}\gamma_{1,3}\alpha_{y}\alpha_{z}P(zxy) - \gamma_{0,1}\gamma_{1,2}\gamma_{1,3}\alpha_{x}\alpha_{y}\alpha_{z}P(zxy)
\end{array}

summing again a few terms we end up with

\begin{array}{}
P(\lambda_{y,x} = 1\vert\{x,y,z\}) = &(\gamma_{0,2}P(xyz) + \gamma_{0,3}(P(xzy) + P(zxy))\alpha_{y} - (\gamma_{0,1}\gamma_{0,2}P(xyz) + \gamma_{0,1}\gamma_{0,3}P(xzy) + \gamma_{0,2}\gamma_{0,3}P(zxy))\alpha_{x}\alpha_{y} - \\
&((\gamma_{0,1}\gamma_{0,3} - \gamma_{0,1}\gamma_{1,3})P(zxy) + \gamma_{0,2}\gamma_{0,3}P(xzy) - \gamma_{0,2}\gamma_{2,3}P(xyz))\alpha_{y}\alpha_{z} + \\
&(\gamma_{0,1}\gamma_{0,2}\gamma_{0,3}P(xzy) - \gamma_{0,1}\gamma_{0,2}\gamma_{2,3}P(xyz) + (\gamma_{0,1}\gamma_{0,2}\gamma_{0,3} - \gamma_{0,1}\gamma_{1,2}\gamma_{1,3})P(zxy))\alpha_{x}\alpha_{y}\alpha_{z}
\end{array}

## Y skipped and X clicked

Binary random variable $\lambda_{x,y}$ is 1 only if the document $y$ was presented above document $x$ and the user clicked on document $x$. The following table show ranking-click paterns consistent with $\lambda_{x,y} = 1$:

$$
\begin{array}{|c|c|l|}
\hline\hline
yxz & 010 & (1 - \gamma_{0,1}\alpha_{y}) \cdot \gamma_{0,2}\alpha_{x} \cdot (1 - \gamma_{2,3}\alpha_{z})\\
yxz & 011 & (1 - \gamma_{0,1}\alpha_{y}) \cdot \gamma_{0,2}\alpha_{x} \cdot \gamma_{2,3}\alpha_{z}\\
\hline
yzx & 001 & (1 - \gamma_{0,1}\alpha_{y}) \cdot (1 - \gamma_{0,2}\alpha_{z}) \cdot \gamma_{0,3}\alpha_{x}\\
yzx & 011 & (1 - \gamma_{0,1}\alpha_{y}) \cdot \gamma_{0,2}\alpha_{z} \cdot \gamma_{2,3}\alpha_{x}\\
\hline
zyx & 001 & (1 - \gamma_{0,1}\alpha_{z}) \cdot (1 - \gamma_{0,2}\alpha_{y}) \cdot \gamma_{0,3}\alpha_{x}\\
zyx & 101 & \gamma_{0,1}\alpha_{z} \cdot (1 - \gamma_{1,2}\alpha_{y}) \cdot \gamma_{1,3}\alpha_{x}\\
\hline\hline
\end{array}
$$

Cosidering that each ranking (permutation) has the same probability of being shown, we get the following expression for $\lambda_{x,y}$:

\begin{array}{}
P(\lambda_{x,y} = 1\vert\{x,y,z\}) = &\gamma_{0,2}\alpha_{x} - \gamma_{0,1}\gamma_{0,2}\alpha_{x}\alpha_{y} + \gamma_{0,3}\alpha_{x} - \gamma_{0,2}\gamma_{0,3}\alpha_{x}\alpha_{z} - \gamma_{0,1}\gamma_{0,3}\alpha_{y}\alpha_{x} + \\
&\gamma_{0,1}\gamma_{0,2}\gamma_{0,3}\alpha_{x}\alpha_{y}\alpha_{z} + \gamma_{0,2}\gamma_{2,3}\alpha_{x}\alpha_{z} - \gamma_{0,1}\gamma_{0,2}\gamma_{2,3}\alpha_{x}\alpha_{y}\alpha_{z} + \gamma_{0,3}\alpha_{x} - \gamma_{0,2}\gamma_{0,3}\alpha_{x}\alpha_{y} - \\
&\gamma_{0,1}\gamma_{0,3}\alpha_{x}\alpha_{z} + \gamma_{0,1}\gamma_{0,2}\gamma_{0,3}\alpha_{x}\alpha_{y}\alpha_{z} + \gamma_{0,1}\gamma_{1,3}\alpha_{x}\alpha_{z} - \gamma_{0,1}\gamma_{1,2}\gamma_{1,3}\alpha_{x}\alpha_{y}\alpha_{z} 
\end{array}

by summing up a few terms in above expression we get

\begin{array}{}
P(\lambda_{x,y} = 1\vert\{x,y,z\}) = &(\gamma_{0,2} + 2\gamma_{0,3})\alpha_{x} - (\gamma_{0,1}\gamma_{0,2} + \gamma_{0,1}\gamma_{0,3} + \gamma_{0,2}\gamma_{0,3})\alpha_{x}\alpha_{y} - (\gamma_{0,1}\gamma_{0,3} + \gamma_{0,2}\gamma_{0,3} - \gamma_{0,2}\gamma_{2,3} - \gamma_{0,1}\gamma_{1,3})\alpha_{x}\alpha_{z} + \\
&(\gamma_{0,1}\gamma_{0,2}\gamma_{0,3} - \gamma_{0,1}\gamma_{0,2}\gamma_{2,3} + \gamma_{0,1}\gamma_{0,2}\gamma_{0,3} - \gamma_{0,1}\gamma_{1,2}\gamma_{1,3})\alpha_{x}\alpha_{y}\alpha_{z}
\end{array}

In more generality we may consider the case in which the rankings are sampled from a non-uniform distribution $P(xyz)$ which denotes the probability of seeing the permutation $xyz$.

\begin{array}{}
P(\lambda_{x,y} = 1\vert\{x,y,z\}) = &\gamma_{0,2}\alpha_{x}P(yxz) - \gamma_{0,1}\gamma_{0,2}\alpha_{x}\alpha_{y}P(yxz) + \gamma_{0,3}\alpha_{x}P(yzx) - \gamma_{0,2}\gamma_{0,3}\alpha_{x}\alpha_{z}P(yzx) - \gamma_{0,1}\gamma_{0,3}\alpha_{x}\alpha_{y}P(yzx) + \\
&\gamma_{0,1}\gamma_{0,2}\gamma_{0,3}\alpha_{x}\alpha_{y}\alpha_{z}P(yzx) + \gamma_{0,2}\gamma_{2,3}\alpha_{x}\alpha_{z}P(yzx) - \gamma_{0,1}\gamma_{0,2}\gamma_{2,3}\alpha_{x}\alpha_{y}\alpha_{z}P(yzx) + \gamma_{0,3}\alpha_{x}P(zyx) - \gamma_{0,2}\gamma_{0,3}\alpha_{x}\alpha_{y}P(zyx) - \\
&\gamma_{0,1}\gamma_{0,3}\alpha_{x}\alpha_{z}P(zyx) + \gamma_{0,1}\gamma_{0,2}\gamma_{0,3}\alpha_{x}\alpha_{y}\alpha_{z}P(zyx) + \gamma_{0,1}\gamma_{1,3}\alpha_{x}\alpha_{z}P(zyx) - \gamma_{0,1}\gamma_{1,2}\gamma_{1,3}\alpha_{x}\alpha_{y}\alpha_{z}P(zyx)
\end{array}

summing again a few terms we end up with

\begin{array}{}
P(\lambda_{x,y} = 1\vert\{x,y,z\}) = &(\gamma_{0,2}P(yxz) + \gamma_{0,3}(P(yzx) + P(zyx))\alpha_{x} - (\gamma_{0,1}\gamma_{0,2}P(yxz) + \gamma_{0,1}\gamma_{0,3}P(yzx) + \gamma_{0,2}\gamma_{0,3}P(zyx))\alpha_{x}\alpha_{y} - \\
&((\gamma_{0,1}\gamma_{0,3} - \gamma_{0,1}\gamma_{1,3})P(zyx) + \gamma_{0,2}\gamma_{0,3}P(yzx) - \gamma_{0,2}\gamma_{2,3}P(yxz))\alpha_{x}\alpha_{z} + \\
&(\gamma_{0,1}\gamma_{0,2}\gamma_{0,3}P(yzx) - \gamma_{0,1}\gamma_{0,2}\gamma_{2,3}P(yxz) + (\gamma_{0,1}\gamma_{0,2}\gamma_{0,3} - \gamma_{0,1}\gamma_{1,2}\gamma_{1,3})P(zyx))\alpha_{x}\alpha_{y}\alpha_{z}
\end{array}

## Formula for Differences of Lambdas

Considering a fixed triplet of documents \{x, y, z\}, than the difference in $\lambda$s for pair $(x, y)$ is

$$
P(\lambda_{x,y} = 1\vert\{x,y,z\}) - P(\lambda_{y,x} = 1\vert\{x,y,z\}) = \left[\gamma_{0,2} + 2\gamma_{0,3} - \alpha_{z}(\gamma_{0,1}\gamma_{0,3} + \gamma_{0,2}\gamma_{0,3} - \gamma_{0,2}\gamma_{2,3} - \gamma_{0,1}\gamma_{1,3})\right](\alpha_{x} - \alpha_{y})
$$

which leads to the following definition of $\Lambda_{x,y}$

\begin{align}
\Lambda_{x,y} &= \sum_{z\in D\setminus{\{x,y}\}}\frac{\left(P(\lambda_{x,y} = 1\vert\{x,y,z\}) - P(\lambda_{y,x} = 1\vert\{x,y,z\})\right)}{\vert D\vert\choose3}\\
&= \frac{\left[\gamma_{0,2} + 2\gamma_{0,3} - \left(\sum_{d\in D\setminus{\{x,y}\}}\alpha_{d}\right)(\gamma_{0,1}\gamma_{0,3} + \gamma_{0,2}\gamma_{0,3} - \gamma_{0,2}\gamma_{2,3} - \gamma_{0,1}\gamma_{1,3})\right](\alpha_{x} - \alpha_{y})}{\vert D\vert\choose3}\\
\end{align}

**<font color='red'>BEWARE:</font>** The above result holds only for rankings sampled uniformly at random.

In [ ]:
import cPickle as pickle
import contextlib

@contextlib.contextmanager
def printoptions(*args, **kwargs):
    original = np.get_printoptions()
    np.set_printoptions(*args, **kwargs)
    yield 
    np.set_printoptions(**original)

with open('./data/model_query_uniform_lambdas_v2_collection_c3.pkl') as ifile:
    MQD = pickle.load(ifile)

alphas = MQD['UBM']['2548']['model'].p_attraction
gammas = MQD['UBM']['2548']['model'].p_examination
deltas = MQD['UBM']['2548']['relevances']
deltas = deltas[:, None] - deltas[None, :]
viewed_lambdas = MQD['UBM']['2548']['stats'][1000000]['viewed_lambdas'][0]
total_lambdas = MQD['UBM']['2548']['stats'][1000000]['total_lambdas'][0]

def Lambda(i, j, alphas, gammas):
    alphas_sum = alphas.sum() - alphas[i] - alphas[j]
    return (gammas[1, -1] + 2 * gammas[2, -1] - alphas_sum * 
            (gammas[0, -1] * gammas[2, -1] + 
             gammas[1, -1] * gammas[2, -1] - 
             gammas[1, -1] * gammas[2, 1] -
             gammas[0, -1] * gammas[2, 0])) * (alphas[i] - alphas[j]) / 120.

true_lambdas = np.array([[Lambda(i, j, alphas, gammas) for j in range(10)] for i in range(10)])

In [ ]:
with printoptions(precision=4, linewidth=np.inf):
    print alphas

In [ ]:
with printoptions(precision=4, linewidth=np.inf):
    print gammas

In [ ]:
with printoptions(precision=4, linewidth=np.inf):
    print deltas

In [ ]:
with printoptions(precision=4, linewidth=np.inf):
    print np.nan_to_num(viewed_lambdas / deltas)

In [ ]:
with printoptions(precision=4, linewidth=np.inf):
    print np.nan_to_num(total_lambdas / deltas)

In [ ]:
with printoptions(precision=4, linewidth=np.inf):
    print np.nan_to_num(true_lambdas / deltas)